### ***EDA BASADO EN EL MACHINE LEARNING***

###### **Filtramos los datos**

In [1]:
# Importamos librerias 
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os 
import env

In [2]:
# credenciales del AWS

HOST_DB = env.DB_HOST
USER_DB = env.DB_USER
PASS_DB = env.DB_PASS
NAME_DB = env.DB_NAME

In [ ]:
# Funcion para conectarse a la base de datos

def BD_connection(host, user_db, user_pass, name_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user_db,
            passwd=user_pass,
            database=name_db,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [4]:
# Nos conectamos a la base de datos 
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)

MYSQL DATABASE connection succesful


In [5]:
# verifiquemos la conexion realizando una consulta simple
if connection:
    cursor = connection.cursor()
    cursor.execute("SELECT DATABASE()")
    db = cursor.fetchone()
    print(f"Conectado a la base de datos: {db[0]}")

Conectado a la base de datos: UrbanTransit


In [6]:
# Funcion para obtener las tablas disponibles
def show_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    for table in tables:
        print(table)

In [7]:
# Corremos la query
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)
show_tables(connection)

MYSQL DATABASE connection succesful
('carga_registros',)
('enriched_taxi_data1',)
('enriched_taxi_data2',)
('taxi_fhv_data',)
('taxi_fhv_data1',)
('taxi_zones',)
('temperaturas',)
('trafico',)


In [8]:
# Crear el engine de SQLAlchemy usando la conexion MySQL
def create_sqlalchemy_engine(host_name, user_bd, password_bd, name_bd):
    url = f"mysql+mysqlconnector://{user_bd}:{password_bd}@{host_name}/{name_bd}"
    return create_engine(url)

In [ ]:
# Creamos el engine
engine = create_sqlalchemy_engine(HOST_DB, USER_DB, PASS_DB, NAME_DB)

In [17]:
# Consultar SQL para obtener la tabla 'users_review' y 'sites'
query_data0 = "SELECT * FROM taxi_zones;"
query_data1 = "SELECT * FROM taxi_fhv_data1 WHERE pickup_datetime BETWEEN '2023-01-01' AND '2024-08-31' ORDER BY id"

In [ ]:
# Cargamos los datos en un Dataframe
taxi_zone = pd.read_sql(query_data0, engine)
taxi_green = pd.read_sql(query_data1, engine)

In [12]:
# Creamos un punto de reinicio
df_taxi_zone = pd.DataFrame(taxi_zone)
df_taxi_green = pd.DataFrame(taxi_green)

### Comenzamos con el EDA -- (EXPLORATORY DATA ANALYSIS) -- 

**¿En qué consiste el EDA?**
-   Consiste en investigar y comprender los datos mediante técnicas visuales y estadísticas para descubrir patrones, identificar anomalías, probar hipótesis y extraer información útil.
-  El EDA ayuda a los científicos de datos y analistas a obtener una comprensión inicial de los datos antes de aplicar modelos predictivos o análisis más avanzados.

In [16]:
df_taxi_green

,id,Pickup_datetime,DropOff_datetime,PULocationID,DOLocationID,trip_miles,driver_pay,VendorID,source,trip_time


In [15]:
df_taxi_zone

,LocationID,Borough,Zone,the_geom
0,1,EWR,Newark Airport,MULTIPOLYGON (((-74.18445299999996 40.69499599...
1,2,Queens,Jamaica Bay,MULTIPOLYGON (((-73.82337597260663 40.63898704...
2,3,Bronx,Allerton/Pelham Gardens,MULTIPOLYGON (((-73.84792614099985 40.87134223...
3,4,Manhattan,Alphabet City,MULTIPOLYGON (((-73.97177410965318 40.72582128...
4,5,Staten Island,Arden Heights,MULTIPOLYGON (((-74.17421738099989 40.56256808...
...,...,...,...,...
256,259,Bronx,Woodlawn/Wakefield,MULTIPOLYGON (((-73.85107116191898 40.91037152...
257,260,Queens,Woodside,MULTIPOLYGON (((-73.90175373399988 40.76077547...
258,261,Manhattan,World Trade Center,MULTIPOLYGON (((-74.01332610899988 40.70503078...
259,262,Manhattan,Yorkville East,MULTIPOLYGON (((-73.94383256699986 40.78285908...
